In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
#import pytesseract as tess
import time #for FPS control
from datetime import datetime
import re

In [2]:
OCR = easyocr.Reader(['en','en'], gpu = True)

In [3]:
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'

In [4]:
model = cv2.dnn_DetectionModel(frozen_model,config_file)

#cv2.dnn.readNetFromTensorflow

In [5]:
class_labels = []
filename = 'Labels.txt'

with open(filename, 'rt') as fpt:
    class_labels = fpt.read().rstrip('\n').split('\n') #Create All labels in new line
    #class_labels.append(fpt.read())

In [6]:
#class_labels[3]

In [8]:
#for i in (2,3,5,7):
#    print(class_labels[i])

In [9]:
#DATASET_DIR = "/Users/royku/Sem II Modules/Project Prep/RA/Living Lab Grant/Car Number Plates/SSD_Data"

#import os
#li = os.listdir(DATASET_DIR)

In [10]:
#img = []

#for k in range(0,len(li)):
#    images = cv2.imread('/Users/royku/Sem II Modules/Project Prep/RA/Living Lab Grant/Car Number Plates/SSD_Data/' + li[k])
#    img.append(images)    

In [11]:
#image = img[4]

In [12]:
#plt.imshow(image)

In [13]:
#converted = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#plt.imshow(converted)

### Model

In [7]:
model.setInputSize(540,540)
model.setInputScale(1.0/127) #255/2---->127 which is gray scale
model.setInputMean((127,127,127)) #mobileNet => [-1,1]
model.setInputSwapRB(True)

<dnn_Model 00000174FFBE1870>

In [8]:
# Regex Conditioning

PalphaNum1 = re.compile(r'^[ABCDFJKMNPRTVW][^IOZ]{0,2}[0-9]{1,4}[^IOZ]{0,1}')               #General With Ended Suffix
#PalphaNum2 = re.compile(r'^[ABCDFJKMNPRTVW][^IOZ]{0,2} [0-9]{1,4}')                           #General Without End Suffix
PalphaNum3 = re.compile(r'^[Q][AKBCLMPRST][^QS]{0,2}[0-9]{1,4}[^QSIOZ]{0,1}')               #Sarawak with Ended Suffix
#PalphaNum4 = re.compile(r'^[Q][AKBCLMPRST][^QS]{0,2} [0-9]{1,4}')                             #Sarawak without End Suffix
PalphaNum5 = re.compile(r'^[S][ABDGKLSTU][AB]{0,2}[^IOZQS]{0,2}[0-9]{1,4}[^QSIOZ]{0,1}')    #Sabah with End Suffix
#PalphaNum6 = re.compile(r'^[S][ABDGKLSTU][AB]{0,2}[^IOZQS]{0,2} [0-9]{1,4}')                  #Sabah without End Suffix
PalphaNum7 = re.compile(r'^[H][B][0-9]{1,4}[S][A]')                                         #Shah Alam Taxi 
PalphaNum8 = re.compile(r'^[H][ABCDEJKLMNPQRSTW][A-Z][0-9]{1,4}')                            #Taxi General
PalphaNum9 = re.compile(r'^[L][I][M][O][0-9]{1,4}[WB]{0,2}')                                #Airport LIMO
PalphaNum10 = re.compile(r'^[Z][ADLUZ]{0,2}[0-9]{1,4}')                                      #Military Plates

#Diplomatic Corps, Consular Corps, UN numberp plates & Commemorative Plates not included

In [9]:
# Removing Strings with Spcl Characters

SCregex = re.compile('[a-z@_!#$%^&*.,()''<>?/\|}{~:]')

### Video Detection

In [10]:
#cap = cv2.VideoCapture("Gate3.mp4")
cap = cv2.VideoCapture(1)

#if not cap.isOpened():
#    cap = cv2.VideoCapture(0)
#if not cap.isOpened():
#    raise IOError("Cannot Open Video")
    
fpsLimit = 0.01 # throttle limit for FPS
startTime = time.time()

cdb = {}

while True:
    
    ret, frame = cap.read()
    nowTime = time.time()
    if (int(nowTime - startTime)) > fpsLimit:
        
        try:

            class_index, confidence, bbox = model.detect(frame, confThreshold = 0.5)

            if (len(class_index)!=0):
                for class_ind, conf, box in zip(class_index.flatten(), confidence.flatten(), bbox):
                    #if (class_ind<=80):
                    if (class_ind-1)==2 or (class_ind-1)==3 or (class_ind-1)==5 or (class_ind-1)==7:
                        if conf >=0.5:

                            cv2.imshow('frame Detection', frame)

                            ## Conversion ##
                            converted = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  #Conversion to gray scale
                            filtered = cv2.bilateralFilter(converted, 11, 17, 17) # Reduce Noise / Filtering 
                            boundary = cv2.Canny(filtered,30,200) # Detect Edge #(apertureSize = 3, L2gradient = True)

                            ## Finding Contours ##
                            contour_point  = cv2.findContours(boundary.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                            contour = imutils.grab_contours(contour_point)
                            contour = sorted(contour, key=cv2.contourArea, reverse=True)[:20]

                            ## Finding Edge Points ##
                            edge_points = []  #None
                            for con in contour:
                                #peri = cv2.arcLength(con, True)
                                approx = cv2.approxPolyDP(con, 10, True)
                                if len(approx) == 4:
                                    edge_points.append(approx)
                                    
                            ## Masking ##
                            if edge_points is not None:

                                for i in edge_points:

                                    #mask = np.zeros(converted.shape, np.uint8)
                                    #plate = cv2.drawContours(mask, [i], 0,255, -1)
                                    #plate = cv2.bitwise_and(frame, frame, mask=mask)

                                    (x,y) = np.where(mask==255)
                                    (x1, y1) = (np.min(x), np.min(y))
                                    (x2, y2) = (np.max(x), np.max(y))
                                    plate= converted[x1:x2+1, y1:y2+1]

                                    # Number Plate conversion to Text
                                    output = OCR.readtext(plate)

                                    ## Number Plate with Time ##
                                    if len(output) != 0:

                                        text = output[0][1].replace(' ','')
                                        
                                        if text not in cdb.keys():
                                            if(SCregex.search(text) == None):
                                                if PalphaNum1.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                                                                       
                                                elif PalphaNum3.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                
                                                elif PalphaNum5.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                
                                                elif PalphaNum7.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                    
                                                elif PalphaNum8.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                    
                                                elif PalphaNum9.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                    
                                                elif PalphaNum10.search(text) is not None:
                                                    now = str(datetime.now())
                                                    cdb[text] = now
                                                    cv2.imshow('Plate Detection', plate)
                                                else:
                                                    pass

            startTime = time.time()
        except Exception as e:
            pass
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [18]:
cdb

{}

In [32]:
cdb

{'WC6414T': '2021-06-03 12:45:35.850182'}

In [70]:
cdb

{'WC6414T': '2021-06-30 15:51:43.764115'}

In [93]:
cdb

{}

In [17]:
cdb

{'JHY6595': '2021-06-07 21:19:26.372497'}

In [26]:
cdb

{}

In [50]:
cdb #KL Traffic Data

{'CENTRE': '2021-05-17 21:27:19.014385',
 'Dentist': '2021-05-17 21:27:51.914797',
 'Dentis(': '2021-05-17 21:28:04.972321',
 'Denrist': '2021-05-17 21:28:57.166660',
 'DENTIL': '2021-05-17 21:29:22.492160',
 'Dentisu': '2021-05-17 21:29:33.073755',
 'Dentis': '2021-05-17 21:29:40.409988',
 'AM 2,00': '2021-05-17 21:31:34.025410',
 'CASHLESS': '2021-05-17 21:31:35.481384',
 'RM 3500': '2021-05-17 21:31:50.366793',
 'cASHLESS': '2021-05-17 21:31:50.982754',
 'RM 3.00': '2021-05-17 21:31:53.488217',
 'RM 3,00': '2021-05-17 21:31:53.924052',
 'CASHGSS': '2021-05-17 21:31:58.423198',
 'casHLESS': '2021-05-17 21:32:16.996856',
 'Subscribe': '2021-05-17 21:32:44.927173',
 'CENIRE': '2021-05-17 21:34:38.322898',
 'Car Park': '2021-05-17 21:35:01.061325',
 'PlevePark': '2021-05-17 21:35:28.657261',
 'Ploper': '2021-05-17 21:35:29.804179',
 'P oper': '2021-05-17 21:35:36.697444',
 'PlepePark': '2021-05-17 21:35:40.692660',
 'P %per': '2021-05-17 21:35:58.611239',
 'P Cper': '2021-05-17 21:36:01

In [33]:
cdb #KL Traffic Data

{'AM 2,00': '2021-05-19 11:52:19.439425',
 'AM 300': '2021-05-19 11:52:25.994304',
 'RKY 200': '2021-05-19 11:52:26.719524',
 'AM 3.00': '2021-05-19 11:52:28.342456',
 'RM 2.00': '2021-05-19 11:52:29.132771',
 'RM 3100': '2021-05-19 11:52:30.782968',
 'RM 3.00': '2021-05-19 11:52:31.893188',
 'RM 3,00': '2021-05-19 11:52:31.978953',
 'AM 3,00': '2021-05-19 11:52:32.989403',
 'AM 3500': '2021-05-19 11:52:40.366318',
 'RM 3500': '2021-05-19 11:52:42.012811',
 'BPN 7963': '2021-05-19 11:57:05.132877',
 'BPN 7963]': '2021-05-19 11:57:05.593741',
 'BPM 7963': '2021-05-19 11:57:09.809275',
 'WC 6414 T': '2021-05-19 11:57:41.341060',
 'MC 6414': '2021-05-19 11:57:42.476864',
 'MC 6a14': '2021-05-19 11:57:43.785363',
 'MC 641 T': '2021-05-19 11:57:48.180091',
 'VC 6414 T': '2021-05-19 11:57:48.232143',
 'WC 6414 t': '2021-05-19 11:57:57.892357'}

In [20]:
cdb #Parking 2 Data

{'WD 6493 D': '2021-05-19 14:10:07.483817',
 'WD 6493D': '2021-05-19 14:10:13.519866'}

In [36]:
cdb #KL Traffic Data

{'BPN7963': '2021-05-20 14:42:03.061203',
 'WC6414T': '2021-05-20 14:42:04.674441'}

In [32]:
cdb #TM 1 data

{'VAD 6149': '2021-05-19 19:48:52.251308'}

In [48]:
cdb #Parking 2 Data

{'WD6493D': '2021-05-19 21:12:40.239845'}

In [38]:
cdb #Parking 2 Data

{'WD6493D': '2021-05-20 14:43:55.486755'}

In [34]:
cdb #Traffic Video Capture 1

{'KK5ZI': '2021-05-20 14:38:37.991245',
 'BG65USJ': '2021-05-20 14:39:20.481528',
 'BC65USJ': '2021-05-20 14:39:26.741203',
 'AK64DHYI': '2021-05-20 14:39:28.806504',
 'AK64DKV': '2021-05-20 14:39:31.784068',
 'AK64DHV': '2021-05-20 14:39:36.069118',
 'AK64DMV': '2021-05-20 14:39:40.817372'}

In [40]:
cdb #TM 1 data

{'VAD6149': '2021-05-20 14:46:00.927375'}